### https://pcmap.place.naver.com/place/{place_id}/home
- 역전할머니 : 원본(raw, 기준 데이터)
- 역전할머니2 : 규칙 기반 처리만 수행(지점명 파싱, 결측치/중복 정리), dtype 통일
- 역전할머니3 : 지점명 기준 수동 place_id 매핑완료
- 역전할머니4 : 이후 중복되어 있는 place_id 처리 및 지점명 채우기
            - place_id로 지점명 찾기
            - place_id 수정하기
            - 지번주소로 지점명, place_id 찾기
            - 중복된 컬럼은 뭐가 다른건지 확인
            - place_id 중복 및 지번주소로도 못찾는 경우 삭제

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None) # 전역 옵션 설정

df= pd.read_csv('../소상공인시장진흥공단_상가정보_추가수집데이터_2.csv')

# brand가 '역전할머니' 인것만 출력
역전할머니 = df[df['brand'] == '역전할머니']
역전할머니

# place_id 정수로 변경하기 
# 층정보, 지번부번지 결측값 건들지말기

,상가업소번호,brand,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,시도코드,시도명,시군구코드,시군구명,행정동코드,행정동명,법정동코드,법정동명,지번코드,지번본번지,지번부번지,지번주소,도로명코드,도로명,도로명주소,층정보,경도,위도,place_id,월간 검색량,블로그누적발행,encoded
9481,MA010120220808007750,역전할머니,역전할머니맥주강서,구청점,I2,음식,I211,주점,I21103,생맥주 전문,I56219,기타 주점업,11,서울특별시,11500,강서구,11500591,화곡6동,1150010300,화곡동,1150010300111200013,1120.0,13.0,서울특별시 강서구 화곡동 1120-13,115004145541,서울특별시 강서구 화곡로58길,서울특별시 강서구 화곡로58길 22-3,1,126.852178,37.552714,1.868360e+09,20.0,231.0,강서구 화곡로58길 역전할머니
9483,MA010120220801940737,역전할머니,역전할머니맥주서울,명지대점,I2,음식,I211,주점,I21103,생맥주 전문,I56213,생맥주 전문점,11,서울특별시,11410,서대문구,11410685,홍은2동,1141011800,홍은동,1141011800104090051,409.0,51.0,서울특별시 서대문구 홍은동 409-51,114104136324,서울특별시 서대문구 증가로10길,서울특별시 서대문구 증가로10길 63,1,126.925270,37.579687,1.323813e+09,20.0,70.0,서대문구 증가로10길 역전할머니
9491,MA010120220806381564,역전할머니,역전할머니맥주,NaN,I2,음식,I211,주점,I21103,생맥주 전문,I56219,기타 주점업,11,서울특별시,11560,영등포구,11560670,신길5동,1156013200,신길동,1156013200104450002,445.0,2.0,서울특별시 영등포구 신길동 445-2,115603118014,서울특별시 영등포구 신길로,서울특별시 영등포구 신길로 67,1,126.908128,37.498765,1.311696e+09,20.0,207.0,영등포구 신길로 역전할머니
9499,MA010120220808643509,역전할머니,역전할머니맥주,서울상봉점,I2,음식,I211,주점,I21103,생맥주 전문,I56213,생맥주 전문점,11,서울특별시,11260,중랑구,11260590,상봉2동,1126010200,상봉동,1126010200101050011,105.0,11.0,서울특별시 중랑구 상봉동 105-11,112604118287,서울특별시 중랑구 봉우재로33길,서울특별시 중랑구 봉우재로33길 42,1,127.088287,37.594237,1.650552e+09,20.0,142.0,중랑구 봉우재로33길 역전할머니
9500,MA010120220810536466,역전할머니,역전할머니맥주,오류점,I2,음식,I211,주점,I21103,생맥주 전문,I56211,일반 유흥 주점업,11,서울특별시,11530,구로구,11530770,오류1동,1153010800,오류동,1153010800100340004,34.0,4.0,서울특별시 구로구 오류동 34-4,115304148080,서울특별시 구로구 경인로25길,서울특별시 구로구 경인로25길 16,1,126.844202,37.497175,1.404199e+09,20.0,457.0,구로구 경인로25길 역전할머니
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10337,MA010120220809722905,역전할머니,역전할머니맥주,NaN,I2,음식,I211,주점,I21103,생맥주 전문,I56219,기타 주점업,41,경기도,41430,의왕시,41430540,내손1동,4143010700,내손동,4143010700107380000,738.0,NaN,경기도 의왕시 내손동 738,414303201007,경기도 의왕시 계원대학로,경기도 의왕시 계원대학로 47,1,126.976662,37.379765,1.891684e+09,20.0,36.0,의왕시 계원대학로 역전할머니
10339,MA010120220806804210,역전할머니,역전할머니맥주,교하점,I2,음식,I211,주점,I21103,생맥주 전문,I56213,생맥주 전문점,41,경기도,41480,파주시,41480590,운정5동,4148011300,동패동,4148011300117010000,1701.0,NaN,경기도 파주시 동패동 1701,414803206084,경기도 파주시 책향기로,경기도 파주시 책향기로 423,1,126.718930,37.724920,1.098387e+09,20.0,59.0,파주시 책향기로 역전할머니
10340,MA010120220811785419,역전할머니,역전할머니맥주김량장점,NaN,I2,음식,I211,주점,I21103,생맥주 전문,I56219,기타 주점업,41,경기도,41461,용인시 처인구,41461510,중앙동,4146110100,김량장동,4146110100102540055,254.0,55.0,경기도 용인시 처인구 김량장동 254-55,414613203017,경기도 용인시 처인구 금령로,경기도 용인시 처인구 금령로 79-4,1,127.205012,37.235532,1.228768e+09,20.0,77.0,처인구 금령로 역전할머니
10341,MA010120220803678522,역전할머니,역전할머니맥주고양,신원점,I2,음식,I211,주점,I21103,생맥주 전문,I56213,생맥주 전문점,41,경기도,41281,고양시 덕양구,41281520,원신동,4128110300,신원동,4128110300106340002,634.0,2.0,경기도 고양시 덕양구 신원동 634-2,412813194072,경기도 고양시 덕양구 신원로,경기도 고양시 덕양구 신원로 12-11,1,126.890880,37.662641,1.091472e+09,20.0,60.0,덕양구 신원로 역전할머니


In [2]:
역전할머니.info()

<class 'pandas.core.frame.DataFrame'>
Index: 344 entries, 9481 to 10345
Data columns (total 34 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   상가업소번호     344 non-null    object 
 1   brand      344 non-null    object 
 2   상호명        344 non-null    object 
 3   지점명        190 non-null    object 
 4   상권업종대분류코드  344 non-null    object 
 5   상권업종대분류명   344 non-null    object 
 6   상권업종중분류코드  344 non-null    object 
 7   상권업종중분류명   344 non-null    object 
 8   상권업종소분류코드  344 non-null    object 
 9   상권업종소분류명   344 non-null    object 
 10  표준산업분류코드   344 non-null    object 
 11  표준산업분류명    344 non-null    object 
 12  시도코드       344 non-null    int64  
 13  시도명        344 non-null    object 
 14  시군구코드      344 non-null    int64  
 15  시군구명       344 non-null    object 
 16  행정동코드      344 non-null    int64  
 17  행정동명       344 non-null    object 
 18  법정동코드      344 non-null    int64  
 19  법정동명       344 non-null    object 
 20  지번코드      

In [3]:
# 타입변환 .astype(타입)
# .astype(타입) => 결측값 있으면 오류남

# 소숫점 제거를 위해 int로 변환
역전할머니['place_id'] = 역전할머니['place_id'].astype('int')
역전할머니['place_id']

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_6484\447851192.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  역전할머니['place_id'] = 역전할머니['place_id'].astype('int')


9481     1868359886
9483     1323812622
9491     1311695977
9499     1650551571
9500     1404198758
            ...    
10337    1891683756
10339    1098387318
10340    1228767540
10341    1091472254
10345    1470294699
Name: place_id, Length: 344, dtype: int64

In [4]:
역전할머니.info()

# 30  place_id   344 non-null    int64 -> place_id dtype 변환 확인 완료

<class 'pandas.core.frame.DataFrame'>
Index: 344 entries, 9481 to 10345
Data columns (total 34 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   상가업소번호     344 non-null    object 
 1   brand      344 non-null    object 
 2   상호명        344 non-null    object 
 3   지점명        190 non-null    object 
 4   상권업종대분류코드  344 non-null    object 
 5   상권업종대분류명   344 non-null    object 
 6   상권업종중분류코드  344 non-null    object 
 7   상권업종중분류명   344 non-null    object 
 8   상권업종소분류코드  344 non-null    object 
 9   상권업종소분류명   344 non-null    object 
 10  표준산업분류코드   344 non-null    object 
 11  표준산업분류명    344 non-null    object 
 12  시도코드       344 non-null    int64  
 13  시도명        344 non-null    object 
 14  시군구코드      344 non-null    int64  
 15  시군구명       344 non-null    object 
 16  행정동코드      344 non-null    int64  
 17  행정동명       344 non-null    object 
 18  법정동코드      344 non-null    int64  
 19  법정동명       344 non-null    object 
 20  지번코드      

In [5]:
# place_id 중복 여부 확인
print(역전할머니['place_id'].nunique(), len(역전할머니))

# place_id 중복된 갯수 확인
print("place_id 중복:", 역전할머니['place_id'].duplicated().sum())

309 344
place_id 중복: 35


In [6]:
# 중복된 place_id 리스트
역전할머니.loc[
    역전할머니['place_id'].duplicated(keep=False),
    'place_id'
].unique()

array([1311695977, 1617580186, 1059948798, 1215644969, 1862325808,
       1219066886, 1298381290, 1724520201, 1726119908, 1864336491,
       1081996780, 1004012665, 1883160911, 1924330846, 1047858386,
       1387072005, 1202988463, 1564245185, 1582135442, 1269638080,
       1336214521, 1942532632, 1446761157, 1700121235, 1280687010,
       1416268607, 1618620966, 1526986047, 1896506249, 1098387318,
       1095383795, 1695640440, 1416990198])

In [7]:
# 중복된 place_id 기준으로 데이터프레임 확인
pd.set_option('display.max_rows', None)

역전할머니.loc[
    역전할머니['place_id'].duplicated(keep=False) # 최초값 포함 중복 나옴
].sort_values('place_id')[['place_id', '상호명', '지점명']]

,place_id,상호명,지점명
9745,1004012665,역전할머니맥주,천호점
9668,1004012665,역전할머니맥주,암사점
9864,1047858386,역전할머니맥주평택,법원점
10056,1047858386,역전할머니맥주평택,동삭점
9628,1059948798,역전할머니맥주,NaN
9523,1059948798,역전할머니맥주,NaN
9607,1081996780,역전할머니맥주,교대점
9680,1081996780,역전할머니맥주,강남역점
10186,1095383795,역전할머니맥주동탄,테크노밸리점
10270,1095383795,역전할머니맥주동탄,영천점


### ***중복처리 순서***
1. **중복 place_id 처리**
    - place_id_fix 사전을 기준으로 중복 place_id 중 올바른 지점만 남기고 나머지는 NaN 처리
    - 잘못된 중복을 제거하기 전에 모든 행을 살려두는 것이 중요

2. **신규 지점 place_id 지정**

    2-1.**처음 데이터에는 중복 place_id가 있음**
    - 예: 1004012665이라는 place_id가 천호점과 암사점이 동시에 존재
    - 즉, 중복 때문에 원래 의도한 지점과 일치하지 않음

    2-2. **place_id를 확인/검색해서 정확한 지점 정보 확보**
    - 예를 들어 암사점은 실제 place_id가 1756212678임 → 기존 중복 place_id와 다름
    - PlaceDetailBase객체 안에 있는 id값
    
    2-3. **정확한 매칭을 위해 기존 place_id를 교체**
    - 기존 place_id가 잘못 들어간 행 → 새로 확인한 정확한 place_id로 변경
    - 이렇게 해야 크롤링이나 분석할 때 실제 매장과 데이터가 일치

    2-4. **데이터 일관성 확보**
    - 모든 지점이 고유 place_id를 갖게 되면
    - 중복/오류로 인한 혼동 제거
    - NaN 또는 잘못된 데이터 없이 분석 가능

3. **place_id dtype 통일**
    - NaN이 존재할 수 있으므로 **nullable integer (Int64)**로 변환

4. **폐점 및 잘못된 중복 행 제거**
    - place_id가 NaN인 행 → 중복 처리 실패 또는 존재하지 않는 place_id
    - 폐점 지점 제거

5. **최종 확인**
    - 모든 place_id가 **정수(Int64)**이고
    - 중복 없이 올바른 지점만 남았는지 확인

In [8]:
# 1004012665 -> 천호점      
# 1047858386 -> 동삭점      
# 1416990198 -> NaN(수원망포역점)
# 1081996780 -> 강남역점
# 1095383795 -> 영천점
# 1098387318 -> 마을점
# 1202988463 -> 진접점
# 1215644969 -> 사당점
# 1219066886 -> 개봉점
# 1269638080 -> NaN(오산궐동점)
# 1280687010 -> NaN
# 1298381290 -> 영등포점
# 1311695977 -> NaN
# 1336214521 -> 중앙점(안산중앙점)
# 1387072005 -> 인계점
# 1416268607 -> 시청역점(부천시청역점)
# 1416990198 -> NaN(수원망포역점)
# 1446761157 -> NaN(군포당동점)
# 1526986047 -> 로데오점(평택고덕로데오점)
# 1564245185 -> NaN
# 1582135442 -> NaN
# 1617580186 -> 까치산점
# 1618620966 -> 반월점
# 1695640440 -> NaN
# 1700121235 -> 태전점
# 1724520201 -> 3가역점
# 1726119908 -> 발산역점
# 1862325808 -> 서울대점(서울대입구역점)
# 1864336491 -> NaN(건대역점)
# 1883160911 -> 사거리점(서울미아사거리점)
# 1896506249 -> 중앙역점(수원광교중앙역점)
# 1924330846 -> NaN(경기광주쌍령점)
# 1942532632 -> NaN(일산웨스턴돔점)

In [9]:
# 기존 역전할머니 복사
# 원본 데이터 보존
역전할머니2 = 역전할머니.copy()

In [10]:
# 현재 place_id 중복 데이터프레임 확인
pd.set_option('display.max_rows', None)

역전할머니2.loc[
   역전할머니2['place_id'].duplicated(keep=False)
].sort_values('place_id')[['place_id', '상호명', '지점명']]

,place_id,상호명,지점명
9745,1004012665,역전할머니맥주,천호점
9668,1004012665,역전할머니맥주,암사점
9864,1047858386,역전할머니맥주평택,법원점
10056,1047858386,역전할머니맥주평택,동삭점
9628,1059948798,역전할머니맥주,NaN
9523,1059948798,역전할머니맥주,NaN
9607,1081996780,역전할머니맥주,교대점
9680,1081996780,역전할머니맥주,강남역점
10186,1095383795,역전할머니맥주동탄,테크노밸리점
10270,1095383795,역전할머니맥주동탄,영천점


### ***현재 상황***
1. 중복 place_id 중 지점명과 동일한 것으로 매핑(역전할머니2)
    - 지점명_fix 확인

2. place_id 새로 추출해서 수정(역전할머니3)
    - place_id dtype 통일
    - 신규 place_id, 지번주소, 도로명주소 일치하는지 확인

3. place_id와 지점명 모두 없는 경우 지번주소 활용하여 place_id, 지점명, 좌표 수정
    - 찾을 지번주소 목록 정의
    - 지번주소 기준 필터링
    - 핵심 컬럼만 확인
    - 지번주소가 고유값인지 확인
    - 지번주소를 기준으로 덮어씌우기
    - 업데이트 후 데이터프레임 확인(위도, 경도 모두 확인)
    - 지점명이 비어있는 행만 대상으로 상호명에서 '역전할머니' 뒤 문자열 추출 -> 지점명으로
    - 빈 문자열은 다시 NaN 처리

4. 수정 완료 후 역전할머니3에 복사

In [11]:
# 중복 place_id 수정 매핑
place_id_fix = {
    1004012665: "천호점",
    1047858386: "동삭점",
    1416990198: None,
    1081996780: "강남역점",
    1095383795: "영천점",
    1098387318: "마을점",
    1202988463: "진접점",
    1215644969: "사당점",
    1219066886: "개봉점",
    1269638080: None,
    1280687010: None,
    1298381290: "영등포점",
    1311695977: None,
    1336214521: "중앙점",
    1387072005: "인계점",
    1416268607: "시청역점",
    1446761157: None,
    1526986047: "로데오점",
    1564245185: None,
    1582135442: None,
    1617580186: "까치산점",
    1618620966: "반월점",
    1695640440: None,
    1700121235: "태전점",
    1724520201: "3가역점",
    1726119908: "발산역점",
    1862325808: "서울대점",
    1864336491: None,
    1883160911: "사거리점",
    1896506249: "중앙역점",
    1924330846: None,
    1942532632: None
}

# place_id_fix를 데이터프레임으로 변환해서 확인하기
fix_df = (
    pd.DataFrame(
        place_id_fix.items(),
        columns=['place_id', '지점명_fix']
    )
)

In [12]:
# 원본 데이터와 join해서 비교하기
# 현재 들어있는 값과 의도한 수정값 한눈에 확인 가능
# place_id 중복 여부 컬럼 생성
check_df = (
    역전할머니2
    .merge(fix_df, on='place_id', how='inner')
    [['place_id', '지점명', '지점명_fix', '지번주소']]
)

# place_id 중복 기준으로 정렬
check_df = check_df.sort_values(by=['place_id'])

display(check_df)


,place_id,지점명,지점명_fix,지번주소
14,1004012665,암사점,천호점,서울특별시 강동구 천호동 288-2
21,1004012665,천호점,천호점,서울특별시 강동구 천호동 413-1
27,1047858386,법원점,동삭점,경기도 평택시 동삭동 697-9
39,1047858386,동삭점,동삭점,경기도 평택시 동삭동 845
16,1081996780,강남역점,강남역점,서울특별시 서초구 서초동 1308-14
11,1081996780,교대점,강남역점,서울특별시 서초구 서초동 1571-27
53,1095383795,테크노밸리점,영천점,경기도 화성시 영천동 874
58,1095383795,영천점,영천점,경기도 화성시 영천동 702-6
52,1098387318,마을점,마을점,경기도 파주시 목동동 935-1
65,1098387318,교하점,마을점,경기도 파주시 동패동 1701


2. place_id 새로 추출해서 수정
    - place_id dtype 통일
    - 신규 place_id, 지번주소, 도로명주소 일치하는지 확인

In [13]:
역전할머니2.columns

Index(['상가업소번호', 'brand', '상호명', '지점명', '상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드',
       '상권업종중분류명', '상권업종소분류코드', '상권업종소분류명', '표준산업분류코드', '표준산업분류명', '시도코드',
       '시도명', '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명', '지번코드',
       '지번본번지', '지번부번지', '지번주소', '도로명코드', '도로명', '도로명주소', '층정보', '경도', '위도',
       'place_id', '월간 검색량', '블로그누적발행', 'encoded'],
      dtype='object')

In [14]:
역전할머니2 = 역전할머니2.drop(columns=['지점명_정렬'], errors='ignore')

In [15]:
# 기존 금별맥주 복사
역전할머니3 = 역전할머니2.copy()

In [16]:
# 지점명 → place_id 매핑
# 총 25개
place_id_map = {
    "암사점": 1756212678,
    "법원점": 1500937003,
    "교대점": 1954680855,
    "테크노밸리점": 1744675487,
    "교하점": 1177597880,
    "진접역점": 1930537898,
    "오산시청점": 1604387325,
    "시청역점": 1416268607,
    "신길점": 1126054619,
    "고잔점": 1429059026,
    "상동점": 1715463219,
    "송내점": 1902214765,
    "영통점": 1264776733,
    "평택고덕점": 1855603068,
    "원흥점": 1851194409,
    "오목천점": 1043971488,
    "화곡역점": 1420076532,
    "반송점": 1626443443,
    "화성봉담점": 1347315519,
    "양벌점": 1667394356,
    "마곡나루점": 1875677589,
    "낙성대점" : 1084106478,
    "서울대점": 1862325808,
    "건대점": 1825323955,
    "아주대점": 1583835753,
}

역전할머니3['place_id'] = (
    역전할머니3['지점명'].map(place_id_map)
    .fillna(역전할머니3['place_id'])
)

In [17]:
# place_id dtype 통일 (nullable integer)
역전할머니3['place_id'] = 역전할머니3['place_id'].astype('Int64')

In [18]:
# place_id 변경한 값 확인
확인용 = (
    역전할머니3
    .loc[
        역전할머니3['지점명'].isin(place_id_map.keys()),
        ['지점명', 'place_id', '지번주소', '도로명주소', '위도', '경도']
    ]
)

print("지점명 -> place_id 매핑 개수: ", 확인용.shape[0])
확인용


지점명 -> place_id 매핑 개수:  25


,지점명,place_id,지번주소,도로명주소,위도,경도
9517,화곡역점,1420076532,서울특별시 강서구 화곡동 1068-23,서울특별시 강서구 화곡로 119,37.540438,126.835106
9559,신길점,1126054619,서울특별시 영등포구 신길동 99-12,서울특별시 영등포구 영등포로 371,37.514225,126.919510
9567,서울대점,1862325808,서울특별시 관악구 봉천동 1611-28,서울특별시 관악구 남부순환로230길 18,37.479276,126.955319
9602,건대점,1825323955,서울특별시 광진구 화양동 45-82,서울특별시 광진구 아차산로25길 13,37.542030,127.066658
9607,교대점,1954680855,서울특별시 서초구 서초동 1571-27,서울특별시 서초구 반포대로28길 73,37.491862,127.012258
9616,낙성대점,1084106478,서울특별시 관악구 봉천동 1688-127,서울특별시 관악구 남부순환로 1947,37.476631,126.965405
9668,암사점,1756212678,서울특별시 강동구 천호동 288-2,서울특별시 강동구 상암로 52,37.549357,127.128939
9678,마곡나루점,1875677589,서울특별시 강서구 마곡동 757,서울특별시 강서구 마곡중앙로 161-8,37.568628,126.827290
9864,법원점,1500937003,경기도 평택시 동삭동 697-9,경기도 평택시 비전2로 22,37.007972,127.096031
9887,오목천점,1043971488,경기도 수원시 권선구 오목천동 314-48,경기도 수원시 권선구 삼천병마로1598번길 32,37.245733,126.965297


In [19]:
역전할머니3.loc[
    역전할머니3['place_id'].duplicated(keep=False),
    ['place_id', '지점명', '지번주소', '도로명주소', '위도', '경도']
].sort_values('place_id')

,place_id,지점명,지번주소,도로명주소,위도,경도
9523,1059948798,NaN,서울특별시 성북구 상월곡동 69-2,서울특별시 성북구 장월로1길 74,37.605597,127.044371
9628,1059948798,NaN,서울특별시 성북구 상월곡동 69-2,서울특별시 성북구 장월로1길 74,37.605597,127.044371
10155,1177597880,야당점,경기도 파주시 야당동 1056,경기도 파주시 경의로 1056,37.711576,126.759536
10339,1177597880,교하점,경기도 파주시 동패동 1701,경기도 파주시 책향기로 423,37.724920,126.718930
9540,1215644969,사당점,서울특별시 관악구 남현동 602-17,서울특별시 관악구 승방4길 20,37.474607,126.980838
9692,1215644969,사당점,서울특별시 관악구 남현동 602-17,서울특별시 관악구 승방4길 20,37.474607,126.980838
9704,1219066886,개봉점,서울특별시 구로구 개봉동 203-12,서울특별시 구로구 남부순환로95길 8-9,37.494291,126.858083
9584,1219066886,NaN,서울특별시 구로구 개봉동 492,서울특별시 구로구 남부순환로 775,37.501576,126.847156
10054,1269638080,시청점,경기도 오산시 원동 811-1,경기도 오산시 운천로 62,37.148014,127.074742
9890,1269638080,NaN,경기도 오산시 궐동 694-6,경기도 오산시 궐동로 18,37.158044,127.056177


In [20]:
# 1177597880 야당점  x
# 1269638080 시청점 -> 1604387325 로 변경
#                  -> 1269638080 오산궐동점으로 지점명 분리
# 1298381290 여의도점 x
# 1347315519 화성봉담 x (10080)
# 1387072005 권선점 x
# 1429059026 한양대점 ->
# 1942532632 NaN -> 일산웨스턴돔점으로 지점명 분리

### ***place_id로 지점명 찾기*** -> 완료
- 1059948798 -> 서울동덕여대점 (서울특별시 성북구 상월곡동 69-2	)
- 1269638080 -> 오산궐동점 (경기도 오산시 궐동 694-6)
- 1298381290 -> 영등포점 (서울특별시 영등포구 영등포동3가 14-1)
- 1311695977 -> 서울신풍역점 (서울특별시 영등포구 신길동 445-2)
- 1347315519 -> 화성봉담2지구점 (경기도 화성시 봉담읍 상리 690-1)
- 1387072005 -> 인계점 (경기도 안산시 단원구 고잔동 770)
- 1429059026 -> 고잔점 (경기도 안산시 단원구 고잔동 770)
- 1446761157 -> 군포당동점 (경기도 군포시 당동 922-2)
- 1724520201 -> 3가역점 (서울특별시 중구 을지로3가 348-7)
- 1862325808 -> 서울대입구역점 (서울특별시 관악구 봉천동 1611-28)
- 1883160911 -> 미아사거리점 (서울특별시 강북구 미아동 55-20)
- 1924330846 -> 경기광주쌍령점 (경기 광주시 쌍령동 353-34)
- 1942532632 -> 일산웨스턴돔점 (경기 고양시 일산동구 장항동 858-1)

### ***place_id 수정하기*** -> 완료 
- 야당점(경기도 파주시 야당동 1056) place_id -> 1046930641

### ***지번주소로 지점명, place_id 찾기*** -> 완료
- 서울특별시 구로구 개봉동 492 -> 서울개봉푸르지오점 -> 1092241695 
- 경기도 오산시 원동 811-1 -> 오산시청점 -> 1604387325
- 서울특별시 영등포구 신길동 480 -> 서울보라매역점 -> 1743181781
- 경기도 안산시 상록구 사동 1570-7 -> 안산한양대점 -> 1095389920
- 경기도 군포시 당정동 974-7 -> 군포당정점 -> 1151781076
- 서울특별시 관악구 봉천동 894-15 -> 서울봉천점 -> 1447876791
- 경기도 성남시 수정구 복정동 685-9 -> 성남동서울대점 -> 1543726258
- 서울특별시 강북구 미아동 211-54 -> 서울미아점 -> 1211085280
- 경기도 광주시 경안동 17-4 -> 경기광주경안점 -> 2073433504

### ***중복 -> 뭐가 다른지 전체 데이터 확인해보기***
- 서울동덕여대점(9523) -> 지점명 NaN으로 되어있음
- 사당점(9540)
- 3가역점(9711)

### ***place_id 중복, 지번주소로도 못찾는 경우***
- 서울특별시 영등포구 여의도동 44-27
- 경기도 화성시 봉담읍 동화리 564
- 경기도 수원시 권선구 권선동 1256-11
- 경기도 고양시 일산동구 장항동 736-1

### ***역전할머니4에서 해야 할 일***
- place_id로 지점명 찾기
- place_id 수정하기
- 지번주소로 지점명, place_id 찾기
- 중복된 컬럼은 뭐가 다른건지 확인
- place_id 중복 및 지번주소로도 못찾는 경우 삭제

In [21]:
역전할머니4 = 역전할머니3.copy()

In [22]:
# place_id → 지점명 매핑 딕셔너리 생성
manual_place_update = {
    1059948798: "서울동덕여대점",
    1269638080: "오산궐동점",
    1298381290: "영등포점",
    1311695977: "서울신풍역점",
    1347315519: "화성봉담2지구점",
    1387072005: "인계점",
    1429059026: "고잔점",
    1446761157: "군포당동점",
    1724520201: "3가역점",
    1862325808: "서울대입구역점",
    1883160911: "미아사거리점",
    1924330846: "경기광주쌍령점",
    1942532632: "일산웨스턴돔점"
}

# place_id 기준으로 지점명 수정
역전할머니4['지점명'] = 역전할머니4['place_id'].map(
    lambda x: manual_place_update.get(x, 역전할머니4.loc[역전할머니4['place_id'] == x, '지점명'].values[0])
)

# 수정 대상 place_id만 추출
수정된_df = 역전할머니4[역전할머니4['place_id'].isin(manual_place_update.keys())]

# place_id 기준으로 정렬
수정된_df = 수정된_df.sort_values('place_id').reset_index(drop=True)

# 데이터프레임 형태로 보기
수정된_df[['place_id', '지점명', '지번주소']]


,place_id,지점명,지번주소
0,1059948798,서울동덕여대점,서울특별시 성북구 상월곡동 69-2
1,1059948798,서울동덕여대점,서울특별시 성북구 상월곡동 69-2
2,1269638080,오산궐동점,경기도 오산시 궐동 694-6
3,1269638080,오산궐동점,경기도 오산시 원동 811-1
4,1298381290,영등포점,서울특별시 영등포구 여의도동 44-27
5,1298381290,영등포점,서울특별시 영등포구 영등포동3가 14-1
6,1311695977,서울신풍역점,서울특별시 영등포구 신길동 480
7,1311695977,서울신풍역점,서울특별시 영등포구 신길동 445-2
8,1347315519,화성봉담2지구점,경기도 화성시 봉담읍 상리 690-1
9,1347315519,화성봉담2지구점,경기도 화성시 봉담읍 동화리 564


In [23]:
# 지번주소로 지점명, place_id 찾은 값 넣기
# 찾을 지번주소 목록 정의
target_jibun = [
    "서울특별시 구로구 개봉동 492",
    "경기도 오산시 원동 811-1",
    "서울특별시 영등포구 신길동 480",
    "경기도 안산시 상록구 사동 1570-7",
    "경기도 군포시 당정동 974-7",
    "서울특별시 관악구 봉천동 894-15",
    "경기도 성남시 수정구 복정동 685-9",
    "서울특별시 강북구 미아동 211-54",
    "경기도 광주시 경안동 17-4"
]

In [24]:
# 지번주소 기준 필터링
check_df = 역전할머니4[역전할머니4['지번주소'].isin(target_jibun)]

In [25]:
# 핵심 컬럼만 확인
check_df[['지점명', '지번주소', '도로명주소', '위도', '경도']]

,지점명,지번주소,도로명주소,위도,경도
9584,NaN,서울특별시 구로구 개봉동 492,서울특별시 구로구 남부순환로 775,37.501576,126.847156
9715,미아사거리점,서울특별시 강북구 미아동 211-54,서울특별시 강북구 솔매로 119,37.626244,127.025262
9779,서울대입구역점,서울특별시 관악구 봉천동 894-15,서울특별시 관악구 청룡1길 7,37.481327,126.946601
9807,서울신풍역점,서울특별시 영등포구 신길동 480,서울특별시 영등포구 대방천로 255,37.499025,126.919142
9851,경기광주쌍령점,경기도 광주시 경안동 17-4,경기도 광주시 광주대로55번길 12,37.411704,127.259111
9916,서울대입구역점,경기도 성남시 수정구 복정동 685-9,경기도 성남시 수정구 복정로 61,37.458377,127.126522
9963,군포당동점,경기도 군포시 당정동 974-7,경기도 군포시 한세로66번길 2,37.348484,126.955420
10054,오산궐동점,경기도 오산시 원동 811-1,경기도 오산시 운천로 62,37.148014,127.074742
10087,고잔점,경기도 안산시 상록구 사동 1570-7,경기도 안산시 상록구 한양대학로 36,37.301170,126.839543


In [26]:
# 지번주소 기준 지점명 + place_id 매핑
jibun_mapping = {
    '서울특별시 구로구 개봉동 492': {'지점명': '서울개봉푸르지오점', 'place_id': 1092241695},
    '경기도 오산시 원동 811-1': {'지점명': '오산시청점', 'place_id': 1604387325},
    '서울특별시 영등포구 신길동 480': {'지점명': '서울보라매역점', 'place_id': 1743181781},
    '경기도 안산시 상록구 사동 1570-7': {'지점명': '안산한양대점', 'place_id': 1095389920},
    '경기도 군포시 당정동 974-7': {'지점명': '군포당정점', 'place_id': 1151781076},
    '서울특별시 관악구 봉천동 894-15': {'지점명': '서울봉천점', 'place_id': 1447876791},
    '경기도 성남시 수정구 복정동 685-9': {'지점명': '성남동서울대점', 'place_id': 1543726258},
    '서울특별시 강북구 미아동 211-54': {'지점명': '서울미아점', 'place_id': 1211085280},
    '경기도 광주시 경안동 17-4': {'지점명': '경기광주경안점', 'place_id': 2073433504}
}

In [27]:
# 지번주소 기준으로 지점명 및 place_id 업데이트
for jibun, values in jibun_mapping.items():
    mask = 역전할머니4['지번주소'] == jibun
    for col in ['지점명', 'place_id']:
        역전할머니4.loc[mask, col] = values[col]

In [28]:
# 업데이트 결과 확인 (데이터프레임 형태)
check_jibun = list(jibun_mapping.keys())
역전할머니4.loc[
    역전할머니4['지번주소'].isin(check_jibun),
    ['지번주소', '지점명', 'place_id']
].sort_values('place_id').reset_index(drop=True)

,지번주소,지점명,place_id
0,서울특별시 구로구 개봉동 492,서울개봉푸르지오점,1092241695
1,경기도 안산시 상록구 사동 1570-7,안산한양대점,1095389920
2,경기도 군포시 당정동 974-7,군포당정점,1151781076
3,서울특별시 강북구 미아동 211-54,서울미아점,1211085280
4,서울특별시 관악구 봉천동 894-15,서울봉천점,1447876791
5,경기도 성남시 수정구 복정동 685-9,성남동서울대점,1543726258
6,경기도 오산시 원동 811-1,오산시청점,1604387325
7,서울특별시 영등포구 신길동 480,서울보라매역점,1743181781
8,경기도 광주시 경안동 17-4,경기광주경안점,2073433504


In [29]:
# 지번주소 기준 place_id 수동 수정
update_place_id = {
    '경기도 파주시 야당동 1056': 1046930641  # 야당점
}

for jibun, pid in update_place_id.items():
    mask = 역전할머니4['지번주소'] == jibun
    역전할머니4.loc[mask, 'place_id'] = pid

In [30]:
# 수정 결과 확인
역전할머니4.loc[
    역전할머니4['지번주소'].isin(update_place_id.keys()),
    ['지번주소', '지점명', 'place_id']
]

,지번주소,지점명,place_id
10155,경기도 파주시 야당동 1056,야당점,1046930641


In [31]:
# 지번주소 기준 지점명 + place_id 업데이트
update_jibun = {
    '경기도 파주시 동패동 1701': {'지점명': '교하점', 'place_id': 1177597880}
}

for jibun, values in update_jibun.items():
    mask = 역전할머니4['지번주소'] == jibun
    for col in ['지점명', 'place_id']:
        역전할머니4.loc[mask, col] = values[col]

# 수정 결과 확인
역전할머니4.loc[
    역전할머니4['지번주소'].isin(update_jibun.keys()),
    ['지번주소', '지점명', 'place_id']
]

,지번주소,지점명,place_id
10339,경기도 파주시 동패동 1701,교하점,1177597880


In [32]:
# 중복 place_id 추출
중복_df = 역전할머니4.loc[
    역전할머니4['place_id'].duplicated(keep=False),
    ['place_id', '지점명', '지번주소', '도로명주소', '위도', '경도']
].sort_values('place_id').reset_index(drop=True)

중복_df

,place_id,지점명,지번주소,도로명주소,위도,경도
0,1059948798,서울동덕여대점,서울특별시 성북구 상월곡동 69-2,서울특별시 성북구 장월로1길 74,37.605597,127.044371
1,1059948798,서울동덕여대점,서울특별시 성북구 상월곡동 69-2,서울특별시 성북구 장월로1길 74,37.605597,127.044371
2,1215644969,사당점,서울특별시 관악구 남현동 602-17,서울특별시 관악구 승방4길 20,37.474607,126.980838
3,1215644969,사당점,서울특별시 관악구 남현동 602-17,서울특별시 관악구 승방4길 20,37.474607,126.980838
4,1298381290,영등포점,서울특별시 영등포구 영등포동3가 14-1,서울특별시 영등포구 영중로10길 34,37.518524,126.908952
5,1298381290,영등포점,서울특별시 영등포구 여의도동 44-27,서울특별시 영등포구 의사당대로1길 25,37.520061,126.929843
6,1347315519,화성봉담2지구점,경기도 화성시 봉담읍 동화리 564,경기도 화성시 봉담읍 동화길 51,37.219032,126.955485
7,1347315519,화성봉담2지구점,경기도 화성시 봉담읍 상리 690-1,경기도 화성시 봉담읍 상리2길 55,37.225075,126.946951
8,1387072005,인계점,경기도 수원시 권선구 권선동 1256-11,경기도 수원시 권선구 권광로56번길 27,37.252673,127.030062
9,1387072005,인계점,경기도 수원시 팔달구 인계동 1038-15,경기도 수원시 팔달구 인계로138번길 27,37.265701,127.032457


In [33]:
역전할머니4.head(2)

,상가업소번호,brand,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,시도코드,시도명,시군구코드,시군구명,행정동코드,행정동명,법정동코드,법정동명,지번코드,지번본번지,지번부번지,지번주소,도로명코드,도로명,도로명주소,층정보,경도,위도,place_id,월간 검색량,블로그누적발행,encoded
9481,MA010120220808007750,역전할머니,역전할머니맥주강서,구청점,I2,음식,I211,주점,I21103,생맥주 전문,I56219,기타 주점업,11,서울특별시,11500,강서구,11500591,화곡6동,1150010300,화곡동,1150010300111200013,1120.0,13.0,서울특별시 강서구 화곡동 1120-13,115004145541,서울특별시 강서구 화곡로58길,서울특별시 강서구 화곡로58길 22-3,1,126.852178,37.552714,1868359886,20.0,231.0,강서구 화곡로58길 역전할머니
9483,MA010120220801940737,역전할머니,역전할머니맥주서울,명지대점,I2,음식,I211,주점,I21103,생맥주 전문,I56213,생맥주 전문점,11,서울특별시,11410,서대문구,11410685,홍은2동,1141011800,홍은동,1141011800104090051,409.0,51.0,서울특별시 서대문구 홍은동 409-51,114104136324,서울특별시 서대문구 증가로10길,서울특별시 서대문구 증가로10길 63,1,126.925270,37.579687,1323812622,20.0,70.0,서대문구 증가로10길 역전할머니


In [34]:
역전할머니5 = 역전할머니4.copy()

In [35]:
역전할머니5.head(2)

,상가업소번호,brand,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,시도코드,시도명,시군구코드,시군구명,행정동코드,행정동명,법정동코드,법정동명,지번코드,지번본번지,지번부번지,지번주소,도로명코드,도로명,도로명주소,층정보,경도,위도,place_id,월간 검색량,블로그누적발행,encoded
9481,MA010120220808007750,역전할머니,역전할머니맥주강서,구청점,I2,음식,I211,주점,I21103,생맥주 전문,I56219,기타 주점업,11,서울특별시,11500,강서구,11500591,화곡6동,1150010300,화곡동,1150010300111200013,1120.0,13.0,서울특별시 강서구 화곡동 1120-13,115004145541,서울특별시 강서구 화곡로58길,서울특별시 강서구 화곡로58길 22-3,1,126.852178,37.552714,1868359886,20.0,231.0,강서구 화곡로58길 역전할머니
9483,MA010120220801940737,역전할머니,역전할머니맥주서울,명지대점,I2,음식,I211,주점,I21103,생맥주 전문,I56213,생맥주 전문점,11,서울특별시,11410,서대문구,11410685,홍은2동,1141011800,홍은동,1141011800104090051,409.0,51.0,서울특별시 서대문구 홍은동 409-51,114104136324,서울특별시 서대문구 증가로10길,서울특별시 서대문구 증가로10길 63,1,126.925270,37.579687,1323812622,20.0,70.0,서대문구 증가로10길 역전할머니


In [36]:
# place_id 중복 및 지번주소로도 못찾는 경우 삭제
# 삭제할 지번주소 리스트
delete_jibun = [
    '서울특별시 영등포구 여의도동 44-27',
    '경기도 화성시 봉담읍 동화리 564',
    '경기도 수원시 권선구 권선동 1256-11',
    '경기도 고양시 일산동구 장항동 736-1'
]

In [37]:
# 지번주소 기준으로 해당 행 삭제
역전할머니5 = 역전할머니5[~역전할머니5['지번주소'].isin(delete_jibun)].reset_index(drop=True)

In [38]:
# 삭제 결과 확인
역전할머니5.loc[역전할머니5['지번주소'].isin(delete_jibun)]

,상가업소번호,brand,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,시도코드,시도명,시군구코드,시군구명,행정동코드,행정동명,법정동코드,법정동명,지번코드,지번본번지,지번부번지,지번주소,도로명코드,도로명,도로명주소,층정보,경도,위도,place_id,월간 검색량,블로그누적발행,encoded


In [39]:
# 중복 place_id 추출
중복_df = 역전할머니5.loc[
    역전할머니5['place_id'].duplicated(keep=False),
].sort_values('place_id').reset_index(drop=True)

중복_df

,상가업소번호,brand,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,시도코드,시도명,시군구코드,시군구명,행정동코드,행정동명,법정동코드,법정동명,지번코드,지번본번지,지번부번지,지번주소,도로명코드,도로명,도로명주소,층정보,경도,위도,place_id,월간 검색량,블로그누적발행,encoded
0,MA010120220813052248,역전할머니,역전할머니맥주,서울동덕여대점,I2,음식,I211,주점,I21103,생맥주 전문,I56213,생맥주 전문점,11,서울특별시,11290,성북구,11290725,월곡2동,1129013700,상월곡동,1129013700100690002,69.0,2.0,서울특별시 성북구 상월곡동 69-2,112904121514,서울특별시 성북구 장월로1길,서울특별시 성북구 장월로1길 74,1,127.044371,37.605597,1059948798,20.0,18.0,성북구 장월로1길 역전할머니
1,MA010120220802511878,역전할머니,역전할머니맥주,서울동덕여대점,I2,음식,I211,주점,I21103,생맥주 전문,I56213,생맥주 전문점,11,서울특별시,11290,성북구,11290725,월곡2동,1129013700,상월곡동,1129013700100690002,69.0,2.0,서울특별시 성북구 상월곡동 69-2,112904121514,서울특별시 성북구 장월로1길,서울특별시 성북구 장월로1길 74,2,127.044371,37.605597,1059948798,20.0,18.0,성북구 장월로1길 역전할머니
2,MA010120220805486530,역전할머니,라쿤컴퍼니역전할머니맥주,사당점,I2,음식,I211,주점,I21103,생맥주 전문,I56213,생맥주 전문점,11,서울특별시,11620,관악구,11620630,남현동,1162010300,남현동,1162010300106020017,602.0,17.0,서울특별시 관악구 남현동 602-17,116204160478,서울특별시 관악구 승방4길,서울특별시 관악구 승방4길 20,NaN,126.980838,37.474607,1215644969,20.0,62.0,관악구 승방4길 역전할머니
3,MA0101202305A0005312,역전할머니,역전할머니맥주,사당점,I2,음식,I211,주점,I21103,생맥주 전문,I56111,한식 일반 음식점업,11,서울특별시,11620,관악구,11620630,남현동,1162010300,남현동,1162010300106020017,602.0,17.0,서울특별시 관악구 남현동 602-17,116204160478,서울특별시 관악구 승방4길,서울특별시 관악구 승방4길 20,1,126.980838,37.474607,1215644969,20.0,62.0,관악구 승방4길 역전할머니
4,MA010120220807754416,역전할머니,역전할머니맥주을지로,3가역점,I2,음식,I211,주점,I21103,생맥주 전문,I56213,생맥주 전문점,11,서울특별시,11140,중구,11140605,을지로동,1114015500,을지로3가,1114015500103480007,348.0,7.0,서울특별시 중구 을지로3가 348-7,111403005006,서울특별시 중구 수표로,서울특별시 중구 수표로 52-1,1,126.990039,37.565853,1724520201,20.0,472.0,중구 수표로 역전할머니
5,MA010120220813886549,역전할머니,역전할머니맥주,3가역점,I2,음식,I211,주점,I21103,생맥주 전문,I56213,생맥주 전문점,11,서울특별시,11140,중구,11140605,을지로동,1114015500,을지로3가,1114015500103480007,348.0,7.0,서울특별시 중구 을지로3가 348-7,111403005006,서울특별시 중구 수표로,서울특별시 중구 수표로 52-1,1,126.990039,37.565853,1724520201,20.0,472.0,중구 수표로 역전할머니


In [40]:
# place_id 기준 중복 제거 (첫 번째 행만 유지)
역전할머니5 = (
    역전할머니5
    .loc[~역전할머니5['place_id'].duplicated(keep='first')]
    .reset_index(drop=True)
)

In [41]:
# 중복 검증
print("place_id 중복 검증(0이면 중복 제거 완료):", 역전할머니5['place_id'].duplicated().sum())  # 0이면 중복 제거 완료

place_id 중복 검증(0이면 중복 제거 완료): 0


In [42]:
행개수 = 역전할머니5.shape[0]
print("역전할머니5(정제완료) 행 개수:",행개수)

역전할머니5[['place_id', '지점명', '지번주소', '도로명주소', '위도', '경도']]

# 정제 전 344 -> 정제 후 337

역전할머니5(정제완료) 행 개수: 337


,place_id,지점명,지번주소,도로명주소,위도,경도
0,1868359886,구청점,서울특별시 강서구 화곡동 1120-13,서울특별시 강서구 화곡로58길 22-3,37.552714,126.852178
1,1323812622,명지대점,서울특별시 서대문구 홍은동 409-51,서울특별시 서대문구 증가로10길 63,37.579687,126.925270
2,1311695977,서울신풍역점,서울특별시 영등포구 신길동 445-2,서울특별시 영등포구 신길로 67,37.498765,126.908128
3,1650551571,서울상봉점,서울특별시 중랑구 상봉동 105-11,서울특별시 중랑구 봉우재로33길 42,37.594237,127.088287
4,1404198758,오류점,서울특별시 구로구 오류동 34-4,서울특별시 구로구 경인로25길 16,37.497175,126.844202
5,1054587722,서울방배역점,서울특별시 서초구 방배동 909-4,서울특별시 서초구 방배로 83-9,37.482051,126.996653
6,1573115649,NaN,서울특별시 동대문구 회기동 60-46,서울특별시 동대문구 경희대로1길 7,37.591610,127.051700
7,1407805860,NaN,서울특별시 성북구 장위동 218-32,서울특별시 성북구 월계로 172,37.619273,127.044889
8,1420076532,화곡역점,서울특별시 강서구 화곡동 1068-23,서울특별시 강서구 화곡로 119,37.540438,126.835106
9,1337289813,삼거리점,서울특별시 동작구 대방동 407-29,서울특별시 동작구 여의대방로24길 141,37.499175,126.927951


In [43]:
역전할머니5

,상가업소번호,brand,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,시도코드,시도명,시군구코드,시군구명,행정동코드,행정동명,법정동코드,법정동명,지번코드,지번본번지,지번부번지,지번주소,도로명코드,도로명,도로명주소,층정보,경도,위도,place_id,월간 검색량,블로그누적발행,encoded
0,MA010120220808007750,역전할머니,역전할머니맥주강서,구청점,I2,음식,I211,주점,I21103,생맥주 전문,I56219,기타 주점업,11,서울특별시,11500,강서구,11500591,화곡6동,1150010300,화곡동,1150010300111200013,1120.0,13.0,서울특별시 강서구 화곡동 1120-13,115004145541,서울특별시 강서구 화곡로58길,서울특별시 강서구 화곡로58길 22-3,1,126.852178,37.552714,1868359886,20.0,231.00,강서구 화곡로58길 역전할머니
1,MA010120220801940737,역전할머니,역전할머니맥주서울,명지대점,I2,음식,I211,주점,I21103,생맥주 전문,I56213,생맥주 전문점,11,서울특별시,11410,서대문구,11410685,홍은2동,1141011800,홍은동,1141011800104090051,409.0,51.0,서울특별시 서대문구 홍은동 409-51,114104136324,서울특별시 서대문구 증가로10길,서울특별시 서대문구 증가로10길 63,1,126.925270,37.579687,1323812622,20.0,70.00,서대문구 증가로10길 역전할머니
2,MA010120220806381564,역전할머니,역전할머니맥주,서울신풍역점,I2,음식,I211,주점,I21103,생맥주 전문,I56219,기타 주점업,11,서울특별시,11560,영등포구,11560670,신길5동,1156013200,신길동,1156013200104450002,445.0,2.0,서울특별시 영등포구 신길동 445-2,115603118014,서울특별시 영등포구 신길로,서울특별시 영등포구 신길로 67,1,126.908128,37.498765,1311695977,20.0,207.00,영등포구 신길로 역전할머니
3,MA010120220808643509,역전할머니,역전할머니맥주,서울상봉점,I2,음식,I211,주점,I21103,생맥주 전문,I56213,생맥주 전문점,11,서울특별시,11260,중랑구,11260590,상봉2동,1126010200,상봉동,1126010200101050011,105.0,11.0,서울특별시 중랑구 상봉동 105-11,112604118287,서울특별시 중랑구 봉우재로33길,서울특별시 중랑구 봉우재로33길 42,1,127.088287,37.594237,1650551571,20.0,142.00,중랑구 봉우재로33길 역전할머니
4,MA010120220810536466,역전할머니,역전할머니맥주,오류점,I2,음식,I211,주점,I21103,생맥주 전문,I56211,일반 유흥 주점업,11,서울특별시,11530,구로구,11530770,오류1동,1153010800,오류동,1153010800100340004,34.0,4.0,서울특별시 구로구 오류동 34-4,115304148080,서울특별시 구로구 경인로25길,서울특별시 구로구 경인로25길 16,1,126.844202,37.497175,1404198758,20.0,457.00,구로구 경인로25길 역전할머니
5,MA010120220800531287,역전할머니,역전할머니맥주,서울방배역점,I2,음식,I211,주점,I21103,생맥주 전문,I56111,한식 일반 음식점업,11,서울특별시,11650,서초구,11650600,방배1동,1165010100,방배동,1165010100109090004,909.0,4.0,서울특별시 서초구 방배동 909-4,116503121010,서울특별시 서초구 방배로,서울특별시 서초구 방배로 83-9,1,126.996653,37.482051,1054587722,20.0,229.00,서초구 방배로 역전할머니
6,MA010120220806451265,역전할머니,역전할머니맥주,NaN,I2,음식,I211,주점,I21103,생맥주 전문,I56219,기타 주점업,11,서울특별시,11230,동대문구,11230710,회기동,1123010800,회기동,1123010800100600046,60.0,46.0,서울특별시 동대문구 회기동 60-46,112304115019,서울특별시 동대문구 경희대로1길,서울특별시 동대문구 경희대로1길 7,1,127.051700,37.591610,1573115649,20.0,109.00,동대문구 경희대로1길 역전할머니
7,MA010120220805522576,역전할머니,역전할머니맥주,NaN,I2,음식,I211,주점,I21103,생맥주 전문,I56213,생맥주 전문점,11,서울특별시,11290,성북구,11290760,장위1동,1129013800,장위동,1129013800102180032,218.0,32.0,서울특별시 성북구 장위동 218-32,112903107018,서울특별시 성북구 월계로,서울특별시 성북구 월계로 172,1,127.044889,37.619273,1407805860,20.0,99.00,성북구 월계로 역전할머니
8,MA010120220803876018,역전할머니,역전할머니맥주,화곡역점,I2,음식,I211,주점,I21103,생맥주 전문,I56213,생맥주 전문점,11,서울특별시,11500,강서구,11500560,화곡3동,1150010300,화곡동,1150010300110680023,1068.0,23.0,서울특별시 강서구 화곡동 1068-23,115003005069,서울특별시 강서구 화곡로,서울특별시 강서구 화곡로 119,1,126.835106,37.540438,1420076532,20.0,231.00,강서구 화곡로 역전할머니
9,MA010120220813808545,역전할머니,역전할머니맥주신대방,삼거리점,I2,음식,I211,주점,I21103,생맥주 전문,I56213,생맥주 전문점,11,서울특별시,11590,동작구,11590660,대방동,1159010800,대방동,1159010800104070029,407.0,29.0,서울특별시 동작구 대방동 407-29,115904157551,서울특별시 동작구 여의대방로24길,서울특별시 동작구 여의대방로24길 141,NaN,126.927951,37.499175,1337289813,20.0,148.00,동작구 여의대방로24길 역전할머니


In [44]:
# 역전할머니 정제 완료 데이터 CSV 저장
역전할머니5.to_csv(
    '역전할머니_정제완료.csv',
    index=False,
    encoding='utf-8-sig'
)

### ***place_id 자릿수 기준 추가 정제 + 덮어쓰기***

In [45]:
# 방금 저장한 CSV 다시 로드
# ================================
df = pd.read_csv(
    '역전할머니_정제완료.csv',
    dtype={'place_id': str}
)

before_cnt = len(df)

In [46]:
# place_id 숫자 길이 계산
# ================================
df['place_id_len'] = df['place_id'].str.len()

In [47]:
# 9자리 이하 place_id 제거
# ================================
df_filtered = df[df['place_id_len'] > 9].copy()

after_cnt = len(df_filtered)
deleted_cnt = before_cnt - after_cnt

In [48]:
# 임시 컬럼 제거
# ================================
df_filtered.drop(columns=['place_id_len'], inplace=True)

In [52]:
# 다시 덮어쓰기 저장
# ================================
df_filtered.to_csv(
    '역전할머니_정제완료.csv',
    index=False,
    encoding='utf-8-sig'
)

In [53]:
# 결과 출력
# ================================
print(f"전체 데이터 수: {before_cnt}")
print(f"삭제된 데이터 수 (place_id 9자리 이하): {deleted_cnt}")
print(f"최종 데이터 수: {after_cnt}")
print("✅ 역전할머니_정제완료.csv 최종 덮어쓰기 완료")

전체 데이터 수: 337
삭제된 데이터 수 (place_id 9자리 이하): 10
최종 데이터 수: 327
✅ 역전할머니_정제완료.csv 최종 덮어쓰기 완료


In [4]:
import pandas as pd
df = pd.read_csv('역전할머니_정제완료.csv')


In [8]:
df['행정동명'].value_counts()

행정동명
가산동     3
동탄1동    3
위례동     3
흥도동     2
고덕동     2
       ..
부발읍     1
비전1동    1
영통3동    1
역북동     1
신곡1동    1
Name: count, Length: 291, dtype: int64

In [9]:
df.head(3)

,상가업소번호,brand,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,도로명코드,도로명,도로명주소,층정보,경도,위도,place_id,월간 검색량,블로그누적발행,encoded
0,MA010120220808007750,역전할머니,역전할머니맥주강서,구청점,I2,음식,I211,주점,I21103,생맥주 전문,...,115004145541,서울특별시 강서구 화곡로58길,서울특별시 강서구 화곡로58길 22-3,1.0,126.852178,37.552714,1868359886,20.0,231.0,강서구 화곡로58길 역전할머니
1,MA010120220801940737,역전할머니,역전할머니맥주서울,명지대점,I2,음식,I211,주점,I21103,생맥주 전문,...,114104136324,서울특별시 서대문구 증가로10길,서울특별시 서대문구 증가로10길 63,1.0,126.925270,37.579687,1323812622,20.0,70.0,서대문구 증가로10길 역전할머니
2,MA010120220806381564,역전할머니,역전할머니맥주,서울신풍역점,I2,음식,I211,주점,I21103,생맥주 전문,...,115603118014,서울특별시 영등포구 신길로,서울특별시 영등포구 신길로 67,1.0,126.908128,37.498765,1311695977,20.0,207.0,영등포구 신길로 역전할머니
